In [2]:
import pandas as pd
import numpy as np

In [3]:
DATADIR = "/scratch2/hugo/ERA5"
CLIMSTOR = "/mnt/climstor/ecmwf/era5/raw"
DATERANGEPL = pd.date_range("19590101", "20211231")
DATERANGEML = pd.date_range("19770101", "20211231")

In [4]:
import cdsapi

c = cdsapi.Client()
for year in np.unique(DATERANGEPL.year):
    c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'variable': 'u_component_of_wind',
            'pressure_level': [
                '775', '925',
            ],
            'year': str(year),
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '06:00', '12:00',
                '18:00',
            ],
            'grid': '0.5/0.5',
            'format': 'netcdf',
        },
        f'{DATADIR}/MoreWind/{year}.nc')

2022-12-07 15:30:10,912 INFO Welcome to the CDS
2022-12-07 15:30:10,913 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2022-12-07 15:30:10,942 INFO Request is queued
2022-12-07 15:30:11,971 INFO Request is running
2022-12-07 15:34:29,053 INFO Request is completed
2022-12-07 15:34:29,054 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data9/adaptor.mars.internal-1670423589.6069336-27169-3-220dd853-7431-4f36-bf9c-ff54fae4d632.nc to /scratch2/hugo/ERA5/MoreWind/1959.nc (1.4G)
2022-12-07 15:35:13,981 INFO Download rate 32.2M/s  
2022-12-07 15:35:14,035 INFO Welcome to the CDS
2022-12-07 15:35:14,036 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2022-12-07 15:35:14,065 INFO Request is queued
2022-12-07 15:35:15,092 INFO Request is running
2022-12-07 15:39:32,119 INFO Request is completed
2022-12-07 15:39:32,120 INFO Downloading https:

In [ ]:
## Not sure I need this

c = cdsapi.Client()
for yearline in np.unique(DATERANGEPL.year).reshape((21, 3)):  
    c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'variable': 'potential_vorticity',
            'pressure_level': [
                '100', '150', '200',
                '250', '300', '350',
                '400', '450', '500',
                '550', '600', '650',
                '700', '750', '800',
                '850', '900', '950',
            ],
            'year': [str(year) for year in yearline],
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '06:00', '12:00',
                '18:00',
            ],
            'format': 'netcdf',
        },
        f'{DATADIR}/PV/{yearline[0]}-{yearline[-1]}.nc')

In [ ]:
import cdsapi
import shutil

c = cdsapi.Client()
months = [str(i) for i in range(1, 13)]
years = [str(i) for i in range(1997, 2023)]
varname = "geopotential"
grid = "1.0/1.0"
plevel = "500"
for year in years:
    for month in months:
        if (year, month) in [("1997", f"{i}") for i in range(1, 12)]:
            continue 
        print("year", year, "month", month)
        result = c.service(
            "tool.toolbox.orchestrator.workflow",
            
            params={
                "realm": "user-apps",
                "project": 'app-c3s-daily-era5-statistics',
                "version": "master",
                "kwargs": {
                    "dataset": "reanalysis-era5-pressure-levels",
                    "product_type": "reanalysis",
                    "variable": varname,
                    "statistic": "daily_mean",
                    "year": year,
                    "month" : month,
                    "time_zone": "UTC+00:0",
                    "frequency": "1-hourly",
                    "grid": grid,
                    "area": {"lat": [15, 80], "lon": [-180, 179]},
                    "pressure_level": plevel,
                },
                "workflow_name": "application"
            }
        )
        c.download(result)
        
        filename = f"{DATADIR}/{varname}_{plevel}hPa_{year}{month}_{grid.split('/')[0]}deg.nc"
        shutil.move(result[0]["location"].split("/")[-1], filename)

In [1]:
from ecmwfapi import ECMWFDataServer
server = ECMWFDataServer()
server.retrieve({
    "class": "e4",
    "dataset": "era40",
    "date": "1957-12-01/to/2002-03-31",
    "levelist": "700/775/850/925",
    "levtype": "pl",
    "param": "131.128/132.128",
    "step": "0",
    "stream": "oper",
    "time": "00:00:00/06:00:00/12:00:00/18:00:00",
    "type": "an",
    "target": "/scratch2/hugo/ERA40/UandV.nc",
})

2023-01-04 12:13:45 ECMWF API python library 1.6.3
2023-01-04 12:13:45 ECMWF API at https://api.ecmwf.int/v1
2023-01-04 12:13:46 Welcome Hugo Banderier
2023-01-04 12:13:46 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web+API+FAQ or contact servicedesk@ecmwf.int
2023-01-04 12:13:46 Request submitted
2023-01-04 12:13:46 Request id: 63b55f6a906002fb28784b4d
2023-01-04 12:13:46 Request is submitted
2023-01-04 12:13:53 Request is active
2023-01-04 13:54:59 mars - INFO   - 20230104.125437 - Looking up FDB indexes: 0.048037 second elapsed, 0.042639 second cpu [ATOS FDB BC]
2023-01-04 13:54:59 mars - INFO   - 20230104.125437 - Calling mars on 'marser-core', local port is 36066
2023-01-04 13:54:59 mars - INFO   - 20230104.125438 - Server task is 559 [marser]
2023-01-04 13:54:59 mars - INFO   - 20230104.125438 - Request cost: 992 fields, 49.0978 Mbytes online, nodes: mvr006 [marser]
2023-01-04 13:54:59 mars - INFO   - 20230104.125438 - The efficiency of your requ